In [1]:
import vk
from tqdm import tqdm

In [2]:
from vk_core import Vk
vk_api = Vk()

In [3]:
session = vk.Session()
api = vk.API(session)

In [5]:
def get_many_of_public(group_id):
    """Все сообщества подписчиков сообщества"""
    users = vk_api.get_all("groups.getMembers", {"group_id": group_id})['items']
    data_for_clustering = []
    for i in range(0, len(users), 1000):
        data_for_clustering.extend(api.users.get(
            user_ids=",".join(map(str, users[i:i+1000])),
            fields="sex,age,education,universities,schools,interests,music,movies,bdate,city,country"
        ))
    
    users_publicpages = []
    
    
    for user in tqdm(users):
        # users and communities followed by any user in this public
#         users_publicpages.append(base_request("users.getSubscriptions", {"user_id": user}))
        users_publicpages.append(api.users.getSubscriptions(user_id=user))
        # friends of any user in this public
#         fields = ",".join(["sex", "bdate", "city", "country", "photo_50", "photo_100", "photo_200_orig",
#                     "photo_200", "photo_400_orig", "photo_max", "photo_max_orig", "photo_id", "online",
#                     "online_mobile", "domain", "has_mobile", "contacts", "connections", "site", "education",
#                     "universities", "schools", "can_post", "can_see_all_posts", "can_see_audio",
#                     "can_write_private_message", "status", "last_seen", "relation",
#                     "relatives", "counters", "screen_name", "maiden_name", "timezone", "occupation,activities",
#                     "interests", "music", "movies", "tv", "books", "games", "about", "quotes", "personal",
#                     "friends_status"])
#         friends_users.append(vk.method("friends.get", {"user_id": user, "fields": fields}))

    return data_for_clustering, users_publicpages

In [6]:
a = get_many_of_public(134459481)

100%|██████████| 38/38 [00:02<00:00, 17.40it/s]
